In [4]:
import pandas as pd
import os
import json
import pymysql
from pyecharts.charts import Graph
from pyecharts import options as opts
from pyecharts.globals import ThemeType
import re
from dominate.tags import *
import dominate
import difflib

In [ ]:
db = pymysql.connect(host='localhost',
                     user='',
                     password='',
                     database='public')
cursor = db.cursor()

In [16]:
lang_map = {}
with open("zh_CN.lang", "r", encoding="utf-8") as lang:
    for i in lang:
        if "name" in i or "desc" in i:
            temp = i.split("=", maxsplit=1)
            lang_map[temp[0].replace("gtnh.quest", "")] = temp[1].strip()

In [ ]:
lang_map

{'0.name': '§d§3§2§1§l你的第一个夜晚',
 '0.desc': '你的第一项任务是寻找食物,为夜晚准备避难所. 夜晚是残酷的,因为这个整合包有困难模式的真实黑暗. 现在收集一些食物可能是个不错的主意...%n%n§l至于这本书？你不需要保存这本书,即使没带在身上也可以用"~"打开任务书.§r ...当然先得把那个碍事的键位冲突解决了.%n%n请到Discord服务器:§6https://discord.gg/EXshrPV§r 获取任务书之外的帮助,或是去闲逛聊天也行.请在#role-select频道选择自己的role以获取符合自己电压等级的建议.%n%n我们的官方wiki:§9https://gtnh.miraheze.org/wiki/Main_Page §r 某些机器设备确实非常复杂,仅靠任务书稍显不足,wiki可以提供更加详细的解释,以及图片. 另外,如果你感觉任务书上的指导还不够,可以看一下wiki上的"beginner\'s guides"初学者指南. wiki上还有可选mod列表,适用于那些想要玩点花的,或者换个字体的人.%n%n最后,在这里§dhttp://bit.ly/gtnh-datasheets§r 还有你所需要的所有数据.%n%n最后,如果你的整合包是用§4Curse§r启动,整合包可能无法稳定运行(Curse无法稳定运行整合包内的部分mod),所以当你遇到任何问题,最佳解决方式是先尝试换个启动器,比如§9MultiMC§r或是§aTechnic§r.最新整合包版本会在discord的#announcements频道发布,或带上!download指令.%n%n§3希望在读这一大段话的时候,你没被怪物嚼碎了.%n%n译者注: 汉化包下载地址:§9https://github.com/Kiwi233/Translation-of-GTNH §r%n%nGTNH新手解答群:QQ§686069758§r%n%n关于中文WIKI： %n主页：§9gtnh.huijiwiki.com§r %n参与编辑：§9gtnh.huijiwiki.com/p/111§r%n可添加MOD：§9gtnh.huijiwiki.com/p/386§r%n常见问题解答：§9gtnh.huijiwiki.com/p/240§r%n%n也可以搜索MC模组百科

In [33]:
df

,quest_file_name,quest_id,quest_name,quest_desc,quest_icon,pre_quests,quest_name_zh,quest_desc_zh
0,AdeptThaumaturgy-AAAAAAAAAAAAAAAAAAABBA==.json,260,Adept Thaumaturgy,I have mastered the basics. Master warned me a...,非凡藏宝袋,[252],熟练的神秘使,我已经掌握了基本的魔法知识.师傅给我讲过一些关于禁忌魔法的事情.但是这个听起来挺有前途的.我...
1,AdvancedNodeStab-AAAAAAAAAAAAAAAAAAAFZw==.json,1383,Advanced Node Stabilizer,The advanced node stabilizer acts like a norma...,高级节点稳定器,[276],高级节点稳定器,"高级节点稳定器的作用类似于普通节点稳定器,但具有关键差异. 它仍然会阻止它影响的节点被耗尽,..."
2,AlmostAutomatedE-AAAAAAAAAAAAAAAAAAABKg==.json,298,(Almost) Automated Essentia,"By combining the essentia locus, the essentia ...",源质聚合器,"[2855, 299]",(几乎)自动化源质,结合源质监视和发射功能于一体.用一个镜子和一个聚合器你可以在任何地方来给注魔祭坛提供源质.(...
3,AMagicalBox-AAAAAAAAAAAAAAAAAAAH5A==.json,2020,A Magical Box,This magical box is amazing. No matter how man...,高级末影箱子,[266],神奇的盒子,"末影箱子非常神奇.无论你制造了多少末影箱子,它们看起来都相互连接到同一维度.%n%n我可以在..."
4,AMagicalTank-AAAAAAAAAAAAAAAAAAAH5Q==.json,2021,A Magical Tank,This tank always seems to connect to the same ...,高级末影储罐,[266],神奇的储罐,"这个储罐似乎总是连接到同一个空间,无论你制造了多少末影储罐. 我可以在一个维度将流体通入储..."
...,...,...,...,...,...,...,...,...
3569,TheEraofMultis-AAAAAAAAAAAAAAAAAAALrQ==.json,2989,The Era of Multis,"What does this mean, ""the era of multis""? I've...",强化钨钢机械方块,"[546, 213]",多方块时代,"""多方块时代"",什么意思? 顾名思义,我已经建造了许多多方块机器了!%n%n好吧,没错,但现..."
3570,UltimateDistilla-AAAAAAAAAAAAAAAAAAAL0w==.json,3027,Ultimate Distillation,The Dangote Distillus is not enough for the go...,巨型蒸馏塔,[3026],终极蒸馏,"丹格特蒸馏厂显然还不能满足石油大亨的处理需求.你可能对""巨型""多方块机器有所耳闻,即巨型工业..."
3571,UniversalFuelPow-AAAAAAAAAAAAAAAAAAAL1Q==.json,3029,Universal Fuel Power,This is the most powerful combustion engine th...,通用化学燃料引擎,[3026],通用燃料能源,"这是有史以来最强大的内燃引擎.事实上,它不仅仅是内燃引擎,还是燃气涡轮以及火箭引擎!%n%n..."
3572,UsingAlltheFract-AAAAAAAAAAAAAAAAAAALuQ==.json,3001,Using All the Fractions,After getting large amounts of all the oil fra...,金币,[2988],利用所有组分,"在提炼出石油的所有组分之后,玩家会用它们做什么呢?这些蒸馏出的组分会囤积在储罐或输出仓中,而..."


In [7]:
df = pd.DataFrame(columns=["quest_file_name", "quest_id",
                  "quest_name", "quest_desc", "quest_icon", "pre_quests"])
for quest_line in os.listdir("DefaultQuests/Quests/"):
    for quest in os.listdir(f"DefaultQuests/Quests/{quest_line}/"):
        que = json.load(
            open(f"DefaultQuests/Quests/{quest_line}/{quest}", "r", encoding="utf-8"))
        
        pre_quests = []
        for i in que["preRequisites:9"]:
            pre_quests.append(que["preRequisites:9"][i]["questIDLow:4"])
        
        item_id = "i~" + que["properties:10"]['betterquesting:10']["icon:10"]["id:8"].replace(":", "~") + "~" + str(que["properties:10"]['betterquesting:10']["icon:10"]["Damage:2"])
        cursor.execute(f"select ID, INTERNAL_NAME, LOCALIZED_NAME from item where ID like '%{item_id}%'")
        icon_name = ""
        try:
            icon_name = cursor.fetchall()[0][2].encode(
                "utf-8").decode("unicode_escape")
        except:
            ...
        df.loc[len(df)] = [quest, que["questIDLow:4"], que["properties:10"]
                           ["betterquesting:10"]["name:8"], que["properties:10"]["betterquesting:10"]["desc:8"], icon_name, pre_quests]

counter = 1
for i in df[df["quest_name"] == "AE Spatial Storage"].sort_values("quest_id").index:
    df.loc[i, "quest_name"] = df.loc[i, "quest_name"] + str(counter)
    counter += 1

df["quest_name_zh"] = df.apply(lambda x: lang_map[str(
    x["quest_id"])+".name"] if str(x["quest_id"])+".name" in lang_map.keys() else "", axis=1)
df["quest_desc_zh"] = df.apply(lambda x: lang_map[str(
    x["quest_id"])+".desc"] if str(x["quest_id"])+".desc" in lang_map.keys() else "", axis=1)

counter = 1
for i in df[df["quest_name_zh"] == "AE空间存储"].sort_values("quest_id").index:
    df.loc[i, "quest_name_zh"] = df.loc[i, "quest_name_zh"] + str(counter)
    counter += 1

In [43]:
for questline in os.listdir("DefaultQuests/QuestLines"):
    nodes = []
    links = []
    for quest in os.listdir(f"DefaultQuests/QuestLines/{questline}"):
        if quest == "QuestLine.json":
            continue
        que = json.load(
            open(f"DefaultQuests/QuestLines/{questline}/{quest}", "r", encoding="utf-8"))
        line = df[df["quest_id"] == que["questIDLow:4"]]

        desc = line["quest_desc"].values[0]
        if len(line["quest_desc_zh"].values[0]) != 0:
            desc = line["quest_desc_zh"].values[0]

        temp_list = []
        for i in range(0, len(desc), 20):
            temp_list.append(desc[i : i + 20])

        if os.path.exists(f"html/2.7.0全物品图标/2.7.0/{line["quest_icon"].values[0]}.png"):
            path = f"image://2.7.0全物品图标/2.7.0/{line["quest_icon"].values[0]}.png"
        else:
            path = ""
        
        name = line["quest_name"].values[0]
        if len(line["quest_name_zh"].values[0]) != 0:
            name = line["quest_name_zh"].values[0]

        new_node = {"name": name,
                        "symbolSize": int(que["sizeX:3"]) * 1.3,
                        "symbol": path,
                        "x": (que["x:3"] + int(que["sizeX:3"]) / 2),
                        "y": (que["y:3"] + int(que["sizeX:3"]) / 2),
                        "tooltip": "<br/>".join(temp_list)
                        }
        nodes.append(new_node)
        pre_quests = line["pre_quests"].values[0]
        for quest_id in pre_quests:
            pre_que = df[df["quest_id"] == quest_id]

            pre_name = pre_que["quest_name"].values[0]
            if len(pre_que["quest_name_zh"].values[0]) != 0:
                pre_name = pre_que["quest_name_zh"].values[0]

            links.append({"source": pre_name,
                          "target": name})

    g = (
        Graph(init_opts=opts.InitOpts(width="100%",
              height="900%", bg_color="#f5f0d3"))
        .add("", nodes, links, is_draggable=False, edge_symbol=['circle', 'arrow'], repulsion=0, label_opts=opts.LabelOpts(position="up", is_show=False), gravity=0, layout="none", edge_symbol_size=10)
        .render(f"html/{questline.split("-")[0]}.html")
    )
    

In [42]:
line["quest_desc_zh"].values[0]

'一氧化二氮,N2O,又称氧化亚氮.它能使汽车引擎在短时间内大幅提升输出功率.事实上,在燃烧过程中将其与燃料混合,作为一种额外的氧气来源,可以使燃料消耗速度更快,同时提升释放能量的速度,但这会有"爆震"的风险,使燃料超出额定范围并可能损坏发动机. %n%n为了避免这种情况,需要在高辛烷值汽油中混合一些其他的东西来防止你的发电机爆炸,对吧?幸运的是,这样的选择是存在的,并且会在下个任务呈现.不过现在,你可以专心混合氮气和氧气,以实现一氧化二氮的自动化.氮和氧是蒸馏空气的两个主要产物,所以你可以一步就获得它们,不过目前你可能已经选好了这些气体的来源,所以直接用你已有的就好.'

In [29]:
quest_line_order = []
for line in open("DefaultQuests/QuestLinesOrder.txt").readlines():
    quest_line_order.append(line.split(": ")[1].strip())

In [34]:
a = ul()
    # if "html" in i:
    #     a += li(button(i.split(".html")[0], id=i))
for que_line in quest_line_order:
    q = difflib.get_close_matches(que_line, os.listdir("html"))[0]
    a += li(button(q.split(".html")[0], id=q))
print(a)

<ul>
  <li>
    <button id="AndSoItBegins.html">AndSoItBegins</button>
  </li>
  <li>
    <button id="TipsandTricks.html">TipsandTricks</button>
  </li>
  <li>
    <button id="Tier0StoneAge.html">Tier0StoneAge</button>
  </li>
  <li>
    <button id="Tier05Steam.html">Tier05Steam</button>
  </li>
  <li>
    <button id="Tier1LV.html">Tier1LV</button>
  </li>
  <li>
    <button id="Tier2MV.html">Tier2MV</button>
  </li>
  <li>
    <button id="Tier3HV.html">Tier3HV</button>
  </li>
  <li>
    <button id="Tier4EV.html">Tier4EV</button>
  </li>
  <li>
    <button id="Tier5IV.html">Tier5IV</button>
  </li>
  <li>
    <button id="Tier6LuV.html">Tier6LuV</button>
  </li>
  <li>
    <button id="Tier7ZPM.html">Tier7ZPM</button>
  </li>
  <li>
    <button id="Tier8UV.html">Tier8UV</button>
  </li>
  <li>
    <button id="Tier9UHV.html">Tier9UHV</button>
  </li>
  <li>
    <button id="Tier10UEV.html">Tier10UEV</button>
  </li>
  <li>
    <button id="Tier11UIV.html">Tier11UIV</button>
  </li>
  <li>


In [4]:
str = """
        document.getElementById('%s').addEventListener('click', function () {
            document.getElementById('contentFrame').src = '%s';
        });
"""
for i in os.listdir("html"):
    if "html" in i:
        print(str % (i, "html/" + i))


        document.getElementById('AdeptThaumaturgy.html').addEventListener('click', function () {
            document.getElementById('contentFrame').src = 'html/AdeptThaumaturgy.html';
        });


        document.getElementById('AndSoItBegins.html').addEventListener('click', function () {
            document.getElementById('contentFrame').src = 'html/AndSoItBegins.html';
        });


        document.getElementById('AppliedEnergisti.html').addEventListener('click', function () {
            document.getElementById('contentFrame').src = 'html/AppliedEnergisti.html';
        });


        document.getElementById('BasicAutomation.html').addEventListener('click', function () {
            document.getElementById('contentFrame').src = 'html/BasicAutomation.html';
        });


        document.getElementById('BeeBreeding.html').addEventListener('click', function () {
            document.getElementById('contentFrame').src = 'html/BeeBreeding.html';
        });


        document.getEl